Let's build a simple neural network to classify images from the FashionMNIST dataset.

**1. Import Libraries**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
%%capture
!pip install lightning

In [ ]:
import lightning as L

*Checking for GPU Availability*

This code checks if a CUDA-enabled GPU is available and sets the `device` accordingly. If no GPU is available, it defaults to the CPU.

In [ ]:
# Check if GPU is available
# device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
# if using windows use next code instead
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: mps


**2. Data Preparation**

In [ ]:
# Define a transform to convert images to tensors
transform = transforms.ToTensor()

# Download and load the training data
train_set = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=256, shuffle=True)

# Download and load the test data
test_set = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=256, shuffle=False)

**3. Neural Network Model**

In [ ]:
class NN(L.LightningModule):
    def __init__(self, model, learning_rate):
        super().__init__()
        self.model = model
        self.learning_rate = learning_rate

    def _common_eval(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        loss = F.cross_entropy(y_hat, y)
        return loss

    def training_step(self, batch, batch_idx):
        loss = self._common_eval(batch, batch_idx)
        self.log('train_loss', loss)
        return loss

    def test_step(self, batch, batch_idx):
        loss = self._common_eval(batch, batch_idx)
        self.log('test_loss', loss)

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer


In [ ]:
def model1(n_features, in_channels, dropout, kernel_size, stride, padding, **kwargs):
    return [
        nn.Conv2d(
            in_channels=in_channels,
            out_channels=32,
            kernel_size=kernel_size,
            stride=stride,
            padding=padding,
        ),
        nn.ReLU(),
        nn.Conv2d(
            in_channels=32,
            out_channels=64,
            kernel_size=kernel_size,
            stride=stride,
            padding=padding,
        ),
        nn.ReLU(),
        nn.Flatten(),
        nn.Linear(in_features=24*24*64, out_features=128),
        nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(in_features=128, out_features=n_features)
    ]

model2 = lambda n_features, in_channels, dropout, kernel_size, stride, padding: [
    nn.Conv2d(
        in_channels=in_channels,
        out_channels=64,
        kernel_size=kernel_size,
        stride=stride,
        padding=padding,
    ),
    nn.ReLU(),
    nn.Conv2d(
        in_channels=64,
        out_channels=32,
        kernel_size=kernel_size,
        stride=stride,
        padding=padding,
    ),
    nn.ReLU(),
    nn.Flatten(),
    nn.Linear(in_features=24*24*32, out_features=128),
    nn.ReLU(),
    nn.Dropout(dropout),
    nn.Linear(in_features=128, out_features=n_features)
]

model3 = lambda n_features, in_channels, dropout, kernel_size, stride, padding: [
    nn.Conv2d(
        in_channels=in_channels,
        out_channels=32,
        kernel_size=kernel_size,
        stride=stride,
        padding=padding,
    ),
    nn.ReLU(),
    nn.Conv2d(
        in_channels=32,
        out_channels=64,
        kernel_size=kernel_size,
        stride=stride,
        padding=padding,
    ),
    nn.ReLU(),
    nn.Flatten(),
    nn.Linear(in_features=24*24*64, out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=64),
    nn.ReLU(),
    nn.Dropout(dropout),
    nn.Linear(in_features=64, out_features=n_features)
]

def get_model(model, epochs=None, learning_rate=None, **kwargs):
    return nn.Sequential(*model(**kwargs))

In [ ]:
def evaluate(model, test_loader):
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():  # Disable gradient calculation for evaluation
        for images, labels in test_loader:
            # Move images and labels to the device
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return correct / total

In [ ]:
from sklearn.model_selection import ParameterGrid

param_grid = {
    'learning_rate': [1e-4, 5e-4, 1e-3, 5e-3],
    'model': [
        # model1,
        model2,
        model3,
    ],
    'epochs': [5, 10],
    'n_features': [10],
    'in_channels': [1],
    'dropout': [0.1, 0.25, 0.5],
    'kernel_size': [(3, 3)],
    'stride': [1],
    'padding': [0],
}

grid = ParameterGrid(param_grid)

In [ ]:
import logging
logging.getLogger("lightning.pytorch").setLevel(logging.ERROR)


In [ ]:
results = []

In [ ]:
import time

for params in grid:
    start = time.time()
    model = NN(
        model=get_model(**params),
        learning_rate=params['learning_rate']
    ).to(device)
    print(id(model))
    trainer = L.Trainer(max_epochs=params['epochs'], enable_model_summary=False)
    trainer.fit(model=model, train_dataloaders=train_loader)
    acc = evaluate(model.model.to(device), test_loader)
    end = time.time()
    print(f'Accuracy: {acc} in {end - start} s with params: {params}', flush=True)
    results.append((acc, params))


11260696224


/opt/anaconda3/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
/opt/anaconda3/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=13` in the `DataLoader` to improve performance.


Training: |                                               | 0/? [00:00<?, ?it/s]

Accuracy: 0.8613 in 21.3739972114563 s with params: {'dropout': 0.1, 'epochs': 5, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.0001, 'model': <function <lambda> at 0x2ce4d8ea0>, 'n_features': 10, 'padding': 0, 'stride': 1}
12030482992


Training: |                                               | 0/? [00:00<?, ?it/s]

...

Accuracy: 0.9164 in 33.81909799575806 s with params: {'dropout': 0.5, 'epochs': 10, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.001, 'model': <function <lambda> at 0x2ce4d82c0>, 'n_features': 10, 'padding': 0, 'stride': 1}
5167476384


Training: |                                               | 0/? [00:00<?, ?it/s]

Accuracy: 0.8955 in 30.440689086914062 s with params: {'dropout': 0.5, 'epochs': 10, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.005, 'model': <function <lambda> at 0x2ce4d8ea0>, 'n_features': 10, 'padding': 0, 'stride': 1}
5167592800


Training: |                                               | 0/? [00:00<?, ?it/s]

Accuracy: 0.907 in 33.61581492424011 s with params: {'dropout': 0.5, 'epochs': 10, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.005, 'model': <function <lambda> at 0x2ce4d82c0>, 'n_features': 10, 'padding': 0, 'stride': 1}


In [ ]:
print('\n'.join(map(str, sorted(results, reverse=True, key=lambda x: x[0]))))

(0.9164, {'dropout': 0.5, 'epochs': 10, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.001, 'model': <function <lambda> at 0x2ce4d82c0>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.9163, {'dropout': 0.25, 'epochs': 10, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.0005, 'model': <function <lambda> at 0x2ce4d8ea0>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.9155, {'dropout': 0.25, 'epochs': 10, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.001, 'model': <function <lambda> at 0x2ce4d8ea0>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.9148, {'dropout': 0.25, 'epochs': 5, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.001, 'model': <function <lambda> at 0x2ce4d82c0>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.9139, {'dropout': 0.1, 'epochs': 10, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.001, 'model': <function <lambda> at 0x2ce4d82c0>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.9129, {'dropou

In [ ]:
model1_results = '''(0.9202, {'dropout': 0.25, 'epochs': 10, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.001, 'model': <function model1 at 0x7a6d3cf91990>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.9167, {'dropout': 0.5, 'epochs': 10, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.001, 'model': <function model1 at 0x7a6d3cf91990>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.9166, {'dropout': 0.1, 'epochs': 10, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.001, 'model': <function model1 at 0x7a6d3cf91990>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.9136, {'dropout': 0.25, 'epochs': 10, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.0005, 'model': <function model1 at 0x7a6d3cf91990>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.9118, {'dropout': 0.1, 'epochs': 10, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.0005, 'model': <function model1 at 0x7a6d3cf91990>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.911, {'dropout': 0.1, 'epochs': 5, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.001, 'model': <function model1 at 0x7a6d3cf91990>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.9109, {'dropout': 0.25, 'epochs': 5, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.001, 'model': <function model1 at 0x7a6d3cf91990>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.9108, {'dropout': 0.25, 'epochs': 5, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.005, 'model': <function model1 at 0x7a6d3cf91990>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.9102, {'dropout': 0.5, 'epochs': 10, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.0005, 'model': <function model1 at 0x7a6d3cf91990>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.9083, {'dropout': 0.1, 'epochs': 5, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.005, 'model': <function model1 at 0x7a6d3cf91990>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.9065, {'dropout': 0.5, 'epochs': 5, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.001, 'model': <function model1 at 0x7a6d3cf91990>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.9061, {'dropout': 0.25, 'epochs': 10, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.005, 'model': <function model1 at 0x7a6d3cf91990>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.9053, {'dropout': 0.1, 'epochs': 5, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.0005, 'model': <function model1 at 0x7a6d3cf91990>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.9032, {'dropout': 0.1, 'epochs': 10, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.005, 'model': <function model1 at 0x7a6d3cf91990>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.8989, {'dropout': 0.25, 'epochs': 5, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.0005, 'model': <function model1 at 0x7a6d3cf91990>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.8978, {'dropout': 0.5, 'epochs': 5, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.0005, 'model': <function model1 at 0x7a6d3cf91990>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.8957, {'dropout': 0.1, 'epochs': 10, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.0001, 'model': <function model1 at 0x7a6d3cf91990>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.8926, {'dropout': 0.25, 'epochs': 10, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.0001, 'model': <function model1 at 0x7a6d3cf91990>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.8909, {'dropout': 0.5, 'epochs': 5, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.005, 'model': <function model1 at 0x7a6d3cf91990>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.8884, {'dropout': 0.5, 'epochs': 10, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.0001, 'model': <function model1 at 0x7a6d3cf91990>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.8866, {'dropout': 0.5, 'epochs': 10, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.005, 'model': <function model1 at 0x7a6d3cf91990>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.8732, {'dropout': 0.1, 'epochs': 5, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.0001, 'model': <function model1 at 0x7a6d3cf91990>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.8724, {'dropout': 0.25, 'epochs': 5, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.0001, 'model': <function model1 at 0x7a6d3cf91990>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.8679, {'dropout': 0.5, 'epochs': 5, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.0001, 'model': <function model1 at 0x7a6d3cf91990>, 'n_features': 10, 'padding': 0, 'stride': 1})'''

In [ ]:
model2_and_model3_results = '''(0.9164, {'dropout': 0.5, 'epochs': 10, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.001, 'model': <function <lambda> at 0x2ce4d82c0>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.9163, {'dropout': 0.25, 'epochs': 10, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.0005, 'model': <function <lambda> at 0x2ce4d8ea0>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.9155, {'dropout': 0.25, 'epochs': 10, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.001, 'model': <function <lambda> at 0x2ce4d8ea0>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.9148, {'dropout': 0.25, 'epochs': 5, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.001, 'model': <function <lambda> at 0x2ce4d82c0>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.9139, {'dropout': 0.1, 'epochs': 10, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.001, 'model': <function <lambda> at 0x2ce4d82c0>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.9129, {'dropout': 0.25, 'epochs': 10, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.001, 'model': <function <lambda> at 0x2ce4d82c0>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.9121, {'dropout': 0.1, 'epochs': 10, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.0005, 'model': <function <lambda> at 0x2ce4d8ea0>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.9121, {'dropout': 0.25, 'epochs': 10, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.005, 'model': <function <lambda> at 0x2ce4d82c0>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.9118, {'dropout': 0.5, 'epochs': 10, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.001, 'model': <function <lambda> at 0x2ce4d8ea0>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.9116, {'dropout': 0.1, 'epochs': 5, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.001, 'model': <function <lambda> at 0x2ce4d8ea0>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.9113, {'dropout': 0.1, 'epochs': 10, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.001, 'model': <function <lambda> at 0x2ce4d8ea0>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.9107, {'dropout': 0.25, 'epochs': 5, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.001, 'model': <function <lambda> at 0x2ce4d8ea0>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.91, {'dropout': 0.1, 'epochs': 10, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.0005, 'model': <function <lambda> at 0x2ce4d82c0>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.9099, {'dropout': 0.25, 'epochs': 10, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.0005, 'model': <function <lambda> at 0x2ce4d82c0>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.9093, {'dropout': 0.1, 'epochs': 10, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.005, 'model': <function <lambda> at 0x2ce4d82c0>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.9091, {'dropout': 0.1, 'epochs': 5, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.001, 'model': <function <lambda> at 0x2ce4d82c0>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.9078, {'dropout': 0.5, 'epochs': 10, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.0005, 'model': <function <lambda> at 0x2ce4d82c0>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.907, {'dropout': 0.5, 'epochs': 10, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.005, 'model': <function <lambda> at 0x2ce4d82c0>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.9068, {'dropout': 0.1, 'epochs': 5, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.005, 'model': <function <lambda> at 0x2ce4d82c0>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.9055, {'dropout': 0.1, 'epochs': 5, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.0005, 'model': <function <lambda> at 0x2ce4d8ea0>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.9055, {'dropout': 0.5, 'epochs': 10, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.0005, 'model': <function <lambda> at 0x2ce4d8ea0>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.9043, {'dropout': 0.5, 'epochs': 5, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.001, 'model': <function <lambda> at 0x2ce4d8ea0>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.9031, {'dropout': 0.5, 'epochs': 5, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.001, 'model': <function <lambda> at 0x2ce4d82c0>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.9018, {'dropout': 0.25, 'epochs': 5, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.005, 'model': <function <lambda> at 0x2ce4d82c0>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.9007, {'dropout': 0.25, 'epochs': 5, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.0005, 'model': <function <lambda> at 0x2ce4d8ea0>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.9002, {'dropout': 0.1, 'epochs': 5, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.005, 'model': <function <lambda> at 0x2ce4d8ea0>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.8991, {'dropout': 0.1, 'epochs': 5, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.0005, 'model': <function <lambda> at 0x2ce4d82c0>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.8988, {'dropout': 0.25, 'epochs': 5, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.0005, 'model': <function <lambda> at 0x2ce4d82c0>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.8976, {'dropout': 0.1, 'epochs': 10, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.005, 'model': <function <lambda> at 0x2ce4d8ea0>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.8969, {'dropout': 0.25, 'epochs': 10, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.005, 'model': <function <lambda> at 0x2ce4d8ea0>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.8955, {'dropout': 0.5, 'epochs': 10, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.005, 'model': <function <lambda> at 0x2ce4d8ea0>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.8943, {'dropout': 0.5, 'epochs': 5, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.0005, 'model': <function <lambda> at 0x2ce4d8ea0>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.8934, {'dropout': 0.5, 'epochs': 5, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.0005, 'model': <function <lambda> at 0x2ce4d82c0>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.8923, {'dropout': 0.5, 'epochs': 5, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.005, 'model': <function <lambda> at 0x2ce4d82c0>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.8922, {'dropout': 0.25, 'epochs': 5, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.005, 'model': <function <lambda> at 0x2ce4d8ea0>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.8897, {'dropout': 0.1, 'epochs': 10, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.0001, 'model': <function <lambda> at 0x2ce4d8ea0>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.8873, {'dropout': 0.25, 'epochs': 10, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.0001, 'model': <function <lambda> at 0x2ce4d8ea0>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.8847, {'dropout': 0.5, 'epochs': 10, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.0001, 'model': <function <lambda> at 0x2ce4d82c0>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.8795, {'dropout': 0.25, 'epochs': 10, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.0001, 'model': <function <lambda> at 0x2ce4d82c0>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.8749, {'dropout': 0.1, 'epochs': 10, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.0001, 'model': <function <lambda> at 0x2ce4d82c0>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.874, {'dropout': 0.5, 'epochs': 10, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.0001, 'model': <function <lambda> at 0x2ce4d8ea0>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.8679, {'dropout': 0.25, 'epochs': 5, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.0001, 'model': <function <lambda> at 0x2ce4d8ea0>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.8661, {'dropout': 0.1, 'epochs': 5, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.0001, 'model': <function <lambda> at 0x2ce4d82c0>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.863, {'dropout': 0.25, 'epochs': 5, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.0001, 'model': <function <lambda> at 0x2ce4d82c0>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.8625, {'dropout': 0.5, 'epochs': 5, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.0001, 'model': <function <lambda> at 0x2ce4d8ea0>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.8613, {'dropout': 0.1, 'epochs': 5, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.0001, 'model': <function <lambda> at 0x2ce4d8ea0>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.8554, {'dropout': 0.5, 'epochs': 5, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.0001, 'model': <function <lambda> at 0x2ce4d82c0>, 'n_features': 10, 'padding': 0, 'stride': 1})
(0.1, {'dropout': 0.5, 'epochs': 5, 'in_channels': 1, 'kernel_size': (3, 3), 'learning_rate': 0.005, 'model': <function <lambda> at 0x2ce4d8ea0>, 'n_features': 10, 'padding': 0, 'stride': 1})'''